In [1]:
import sys
import pickle
import pandas as pd
import numpy as np

In [2]:
filename = "co_T0_T1_toshare.pkl"
area_to_minority = "OAtoMinorityData.CSV"

In [13]:
pickle_in = open(filename,"rb")
df1 = pickle.load(pickle_in)
df2 = pd.read_csv(area_to_minority)

/Users/chaiqifei/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
df = pd.merge(df1,df2)

In [15]:
from IPython.display import display
pd.options.display.max_columns = None
df.head()

,CompanyName,CompanyNumber,RegAddressCareOf,RegAddressPOBox,RegAddressAddressLine1,RegAddressAddressLine2,RegAddressPostTown,RegAddressCounty,RegAddressCountry,RegAddressPostCode,CompanyCategory,CompanyStatus_T0,CountryOfOrigin,DissolutionDate,IncorporationDate,AccountsAccountRefDay,AccountsAccountRefMonth,AccountsNextDueDate,AccountsLastMadeUpDate,AccountsAccountCategory,ReturnsNextDueDate,ReturnsLastMadeUpDate,MortgagesNumMortCharges,MortgagesNumMortOutstanding,MortgagesNumMortPartSatisfied,MortgagesNumMortSatisfied,LimitedPartnershipsNumGenPartners,LimitedPartnershipsNumLimPartners,PreviousName_1CONDATE,PreviousName_1CompanyName,PreviousName_2CONDATE,PreviousName_2CompanyName,PreviousName_3CONDATE,PreviousName_3CompanyName,ConfStmtNextDueDate,ConfStmtLastMadeUpDate,nSIC,SIC1,SIC2,SIC3,eCompanyCategory,eCompanyStatus,eCountryOfOrigin,eAccountsAccountCategory,dIncorporationDate,dAccountsNextDueDate,dAccountsLastMadeUpDate,dReturnsNextDueDate,dReturnsLastMadeUpDate,dConfStmtNextDueDate,dConfStmtLastMadeUpDate,dPreviousName_1CONDATE,dPreviousName_2CONDATE,namechanged,namechanged2,pcd,oseast1m,osnrth1m,oa11,cty,ru11ind,oac11,lat,long,imd,country,oac1,oac2,imdu,CompanyStatus_T1,class_1,Unnamed: 0,GeographyCode,MinorityScore
0,'YOUR HAIR' BY TJ LTD,10037712,NaN,NaN,54 WOOD STREET,NaN,LYTHAM ST ANNES,LANCASHIRE,UNITED KINGDOM,FY8 1QG,Private Limited Company,Active,United Kingdom,NaN,02/03/2016,31.0,3.0,31/12/2018,31/03/2017,N/A,30/03/2017,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,15/03/2020,01/03/2017,96020.0,96.0,0.0,20.0,15,2,182,9,2016.092896,2019.000000,2017.246575,2017.243836,NaN,2020.204918,2017.008219,NaN,NaN,0,0,FY81QG,332135.0,428681.0,E00126832,E10000017,C1,2C1,53.750075,-3.030631,2259.0,E,2,2C,2188.464414,Active,False,131169,E00126832,0.185000
1,( THE LION'S DEN (TLD ) LTD.,08315563,NaN,NaN,24 MANCHESTER HOUSE,EAST STREET,LONDON,NaN,NaN,SE17 2DW,Private Limited Company,Active,United Kingdom,NaN,03/12/2012,31.0,12.0,30/09/2018,31/12/2016,DORMANT,31/12/2016,03/12/2015,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,17/12/2019,03/12/2017,59200.0,59.0,2.0,0.0,15,2,182,3,2012.196721,2018.747945,2017.000000,2017.000000,2015.194521,2019.961644,2017.194521,NaN,NaN,0,0,SE172DW,532386.0,178306.0,E00019964,E99999999,A1,3B3,51.488214,-0.094632,4878.0,E,3,3B,4725.688096,Active,False,131564,E00019964,0.809278
2,(A) EMT MEDICAL SERVICES LTD,08439497,NaN,NaN,30 CASTLE HILL,NaN,ECKINGTON,NaN,NaN,S21 4AY,Private Limited Company,Active,United Kingdom,NaN,11/03/2013,30.0,9.0,30/06/2019,30/09/2017,TOTAL EXEMPTION FULL,08/04/2017,11/03/2016,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,25/03/2020,11/03/2017,86900.0,86.0,9.0,0.0,15,2,182,13,2013.841096,2019.495890,2017.747945,2017.591781,2016.841530,2020.232240,2017.841096,NaN,NaN,0,0,S214AY,442860.0,379595.0,E00100145,E10000007,B1,8A2,53.311565,-1.358187,15456.0,E,8,8A,14973.397953,NaN,True,131615,E00100145,0.011331
3,(A) PROPERTY SALES LIMITED,01359445,NaN,NaN,HANOVERGREEN COTTAGE HANOVER,REDMARLEY,GLOUCESTER,NaN,NaN,GL19 3JZ,Private Limited Company,Active,United Kingdom,NaN,22/03/1978,30.0,4.0,31/01/2018,30/04/2016,TOTAL EXEMPTION SMALL,12/04/2017,15/03/2016,9,4,0,5,0,0,NaN,NaN,NaN,NaN,NaN,NaN,29/03/2020,15/03/2017,68100.0,68.0,1.0,0.0,15,2,182,14,1978.221918,2018.084932,2016.330601,2017.926027,2016.204918,2020.243169,2017.202740,NaN,NaN,0,0,GL193JZ,375920.0,232209.0,E00113214,E10000013,F1,1C2,51.987816,-2.352064,17507.0,E,1,1C,16960.357011,Active,False,131618,E00113214,0.024590
4,(ADAM) COOPER HOMES LIMITED,04873008,NaN,NaN,12 MARKET PLACE,MARKET BOSWORTH,NUNEATON,WARWICKSHIRE,NaN,CV13 0LE,Private Limited Company,Active,United Kingdom,NaN,20/08/2003,31.0,8.0,31/05/2018,31/08/2016,TOTAL EXEMPTION SMALL,17/09/2016,20/08/2015,7,7,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,03/09/2019,20/08/2017,43390.0,43.0,3.0,90.0,15,2,182,14,2003.635616,2018.413699,2016.666667,2016.713115,2015.635616,2019.186301,2017.635616,NaN,NaN,0,0,CV130LE,440579.0,303063.0,E00131109,E10000018,D1,5B3,52.623820,-1.401986,27823.0,E,5,5B,26954.247623,Active

In [16]:
#keep only those companies whose T0 status is active
df = df[df['CompanyStatus_T0']=='Active']

In [24]:
features = df[['CompanyNumber', 'oac2',
        'CountryOfOrigin','MortgagesNumMortCharges',
       'MortgagesNumMortOutstanding', 'MortgagesNumMortPartSatisfied', 'LimitedPartnershipsNumGenPartners',
       'LimitedPartnershipsNumLimPartners',  'SIC1',
       'eCompanyCategory', 'eCompanyStatus',
       'eAccountsAccountCategory', 'dIncorporationDate',
       'dAccountsNextDueDate',
       'dReturnsNextDueDate','dConfStmtNextDueDate','namechanged', 'namechanged2', 'class_1','MinorityScore']].copy()
#Remove the rows with missing data 
features.dropna(how='any',inplace=True)

In [25]:
#Create new features(periods)
T0 = 2018+(31+28)/365
T1 = 2019+(31+28)/365
features['dAccountsOutstandingTime'] = T0-features['dAccountsNextDueDate']
features['dConfStmtOutstandingTime'] = T0-features['dConfStmtNextDueDate']
features['dReturnsOutstandingTime'] = T0-features['dReturnsNextDueDate']
features['OperatingTime'] = T1-features['dIncorporationDate']
features.loc[features['dAccountsOutstandingTime']<0, 'dAccountsOutstandingTime'] = 0 
features.loc[features['dConfStmtOutstandingTime']<0, 'dConfStmtOutstandingTime'] = 0 
features.loc[features['dReturnsOutstandingTime']<0, 'dReturnsOutstandingTime'] = 0
features.drop(['dAccountsNextDueDate','dConfStmtNextDueDate','dReturnsNextDueDate','dIncorporationDate'],axis = 1,inplace = True)
features.head()

,CompanyNumber,oac2,CountryOfOrigin,MortgagesNumMortCharges,MortgagesNumMortOutstanding,MortgagesNumMortPartSatisfied,LimitedPartnershipsNumGenPartners,LimitedPartnershipsNumLimPartners,SIC1,eCompanyCategory,eCompanyStatus,eAccountsAccountCategory,namechanged,namechanged2,class_1,MinorityScore,dAccountsOutstandingTime,dConfStmtOutstandingTime,dReturnsOutstandingTime,OperatingTime
0,10037712,2C,United Kingdom,0,0,0,0,0,96.0,15,2,9,0,0,False,0.185000,0.000000,0.0,0.917808,3.068748
1,08315563,3B,United Kingdom,0,0,0,0,0,59.0,15,2,3,0,0,False,0.809278,0.000000,0.0,1.161644,6.964923
2,08439497,8A,United Kingdom,0,0,0,0,0,86.0,15,2,13,0,0,True,0.011331,0.000000,0.0,0.569863,5.320548
3,01359445,1C,United Kingdom,9,4,0,0,0,68.0,15,2,14,0,0,False,0.024590,0.076712,0.0,0.235616,40.939726
4,04873008,5B,United Kingdom,7,7,0,0,0,43.0,15,2,14,0,0,False,0.045455,0.000000,0.0,1.448529,15.526027


In [27]:
features['dConfStmtOutstandingTime'].value_counts()

0.0    1700590
Name: dConfStmtOutstandingTime, dtype: int64

In [28]:
features['CountryOfOrigin'].value_counts()

United Kingdom    1700590
Name: CountryOfOrigin, dtype: int64

In [29]:
features['LimitedPartnershipsNumGenPartners'].value_counts()

0    1700590
Name: LimitedPartnershipsNumGenPartners, dtype: int64

In [30]:
features['eCompanyStatus'].value_counts()

2    1700590
Name: eCompanyStatus, dtype: int64

In [31]:
features['LimitedPartnershipsNumLimPartners'].value_counts()

0    1700590
Name: LimitedPartnershipsNumLimPartners, dtype: int64

In [32]:
features['dAccountsOutstandingTime'].value_counts()

0.000000     1654874
0.076712        8936
0.161644        8510
0.043836        1260
0.128767        1241
0.413699        1178
0.131507        1062
0.046575        1004
0.136986         902
0.147945         788
0.052055         787
0.139726         770
0.054795         741
0.145205         634
0.142466         622
0.002740         604
0.246575         584
0.060274         569
0.013699         567
0.063014         553
0.005479         531
0.021918         525
0.153425         522
0.010959         520
0.008219         518
0.068493         515
0.024658         510
0.030137         498
0.032877         495
0.328767         483
              ...   
2.800000           1
1.372026           1
5.831043           1
1.197163           1
6.717808           1
1.705360           1
0.898630           1
1.150685           1
6.164384           1
1.415742           1
1.391152           1
8.328767           1
1.505906           1
0.386301           1
0.857534           1
3.430137           1
0.578082     

In [33]:
features.drop(['dConfStmtOutstandingTime','CountryOfOrigin','LimitedPartnershipsNumGenPartners','LimitedPartnershipsNumLimPartners','eCompanyStatus'],axis = 1,inplace = True)

In [34]:
features = pd.get_dummies(features, columns=['SIC1', 'oac2','eCompanyCategory','eAccountsAccountCategory'])

In [35]:
features.head()

,CompanyNumber,MortgagesNumMortCharges,MortgagesNumMortOutstanding,MortgagesNumMortPartSatisfied,namechanged,namechanged2,class_1,MinorityScore,dAccountsOutstandingTime,dReturnsOutstandingTime,OperatingTime,SIC1_1.0,SIC1_2.0,SIC1_3.0,SIC1_5.0,SIC1_6.0,SIC1_7.0,SIC1_8.0,SIC1_9.0,SIC1_10.0,SIC1_11.0,SIC1_12.0,SIC1_13.0,SIC1_14.0,SIC1_15.0,SIC1_16.0,SIC1_17.0,SIC1_18.0,SIC1_19.0,SIC1_20.0,SIC1_21.0,SIC1_22.0,SIC1_23.0,SIC1_24.0,SIC1_25.0,SIC1_26.0,SIC1_27.0,SIC1_28.0,SIC1_29.0,SIC1_30.0,SIC1_31.0,SIC1_32.0,SIC1_33.0,SIC1_35.0,SIC1_36.0,SIC1_37.0,SIC1_38.0,SIC1_39.0,SIC1_41.0,SIC1_42.0,SIC1_43.0,SIC1_45.0,SIC1_46.0,SIC1_47.0,SIC1_49.0,SIC1_50.0,SIC1_51.0,SIC1_52.0,SIC1_53.0,SIC1_55.0,SIC1_56.0,SIC1_58.0,SIC1_59.0,SIC1_60.0,SIC1_61.0,SIC1_62.0,SIC1_63.0,SIC1_64.0,SIC1_65.0,SIC1_66.0,SIC1_68.0,SIC1_69.0,SIC1_70.0,SIC1_71.0,SIC1_72.0,SIC1_73.0,SIC1_74.0,SIC1_75.0,SIC1_77.0,SIC1_78.0,SIC1_79.0,SIC1_80.0,SIC1_81.0,SIC1_82.0,SIC1_84.0,SIC1_85.0,SIC1_86.0,SIC1_87.0,SIC1_88.0,SIC1_90.0,SIC1_91.0,SIC1_92.0,SIC1_93.0,SIC1_94.0,SIC1_95.0,SIC1_96.0,SIC1_97.0,SIC1_98.0,SIC1_99.0,oac2_1A,oac2_1B,oac2_1C,oac2_2A,oac2_2B,oac2_2C,oac2_2D,oac2_3A,oac2_3B,oac2_3C,oac2_3D,oac2_4A,oac2_4B,oac2_4C,oac2_5A,oac2_5B,oac2_6A,oac2_6B,oac2_7A,oac2_7B,oac2_7C,oac2_7D,oac2_8A,oac2_8B,oac2_8C,oac2_8D,eCompanyCategory_15,eCompanyCategory_19,eAccountsAccountCategory_0,eAccountsAccountCategory_1,eAccountsAccountCategory_2,eAccountsAccountCategory_3,eAccountsAccountCategory_4,eAccountsAccountCategory_5,eAccountsAccountCategory_6,eAccountsAccountCategory_7,eAccountsAccountCategory_8,eAccountsAccountCategory_9,eAccountsAccountCategory_10,eAccountsAccountCategory_11,eAccountsAccountCategory_12,eAccountsAccountCategory_13,eAccountsAccountCategory_14,eAccountsAccountCategory_15
0,10037712,0,0,0,0,0,False,0.185000,0.000000,0.917808,3.068748,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,08315563,0,0,0,0,0,False,0.809278,0.000000,1.161644,6.964923,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,08439497,0,0,0,0,0,True,0.011331,0.000000,0.569863,5.320548,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,01359445,9,4,0,0,0,False,0.024590,0.076712,0.235616,40.939726,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,04873008,7,7,0,0,0,False,0.045455,0.000000,1.448529,15.526027,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [37]:
pickle.dump(features, open("processed_data.pkl", "wb" ) )